In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow==2.15.0
!pip install tensorflow-addons==0.22.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [ ]:


# Import necessary libraries
import gradio as gr
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import cv2

# Use the InstanceNormalization from tensorflow_addons
InstanceNormalization = tfa.layers.InstanceNormalization

# Load models with custom InstanceNormalization layer
def load_model_with_custom_objects(model_path):
    return load_model(
        model_path,
        custom_objects={"InstanceNormalization": InstanceNormalization}
    )

# Load the generators (horse to zebra, zebra to horse)
g_AB = load_model_with_custom_objects('/content/drive/MyDrive/Colab Notebooks/GeneratorHtoZ_80e.h5')
g_BA = load_model_with_custom_objects('/content/drive/MyDrive/Colab Notebooks/GeneratorZtoH_80e.h5')

# Preprocess image (resize and normalize)
def preprocess_image(img):
    img = cv2.resize(img, (256, 256))
    img = img.astype('float32') / 127.5 - 1.0  # Normalize to [-1, 1]
    return np.expand_dims(img, axis=0)

# Postprocess image (rescale to [0, 255])
def postprocess_image(img_array):
    img_array = (img_array + 1) * 127.5  # Rescale to [0, 255]
    return np.clip(img_array, 0, 255).astype(np.uint8)[0]

# Translate image using the appropriate model
def translate_image(image, model_choice):
    img = preprocess_image(image)
    # Use the correct model based on choice
    if model_choice == "Horse to Zebra":
        translated_img = g_AB.predict(img)
    else:
        translated_img = g_BA.predict(img)
    return postprocess_image(translated_img)

# Gradio UI function
def gradio_interface(image, model_choice):
    translated_img = translate_image(image, model_choice)
    return translated_img

# Styling for Gradio UI
custom_css = """
.container {
    background-color: #f8f9fa;
    border: 2px solid #6c757d;
    border-radius: 10px;
    padding: 20px;
    box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
}

.label {
    font-size: 18px;
    font-weight: bold;
}

.gr-button {
    background-color: #007bff;
    color: white;
    border-radius: 5px;
    font-size: 16px;
}

.gr-button:hover {
    background-color: #0056b3;
}
"""

# Build Gradio Interface
interface = gr.Interface(
    fn=gradio_interface,
    inputs=[gr.Image(type="numpy", label="Upload Image"),
            gr.Radio(choices=["Horse to Zebra", "Zebra to Horse"], label="Select Model")],
    outputs=gr.Image(type="numpy", label="Translated Image"),
    title="CycleGAN Image Translator",
    description="Upload an image of a horse or zebra and select the desired translation.",
    css=custom_css
)

# Launch Interface
interface.launch()


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're u

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab0b585bf6b86cf453.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
